# Imports

In [2]:
import pandas as pd
import numpy as np 
import json 
import os
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
from sklearn.preprocessing import MinMaxScaler

In [3]:
playercard_data = r'.\output\playercard_stats.csv'
playercard_df = pd.read_csv(playercard_data)
playercard_df

,name_index,ranking,win rate,Overall,Offense,Defense,Aggression,Speed,discord_username,playercard_name,img_filepath,rank
0,ix mini,0,0.53,99.00,100.00,58.41,75.58,95.73,Mini#6984,IX Mini,c:/Users/dmarc/OneDrive/Documents/Github/Perso...,NaN
1,smiggy,1,0.58,95.97,96.81,93.02,60.72,61.00,Smiggy#1206,smiggy,c:/Users/dmarc/OneDrive/Documents/Github/Perso...,Grand Champion I
2,shaunch,2,0.61,94.38,81.73,53.93,79.75,86.59,Shaunch#6025,Shaunch,c:/Users/dmarc/OneDrive/Documents/Github/Perso...,Champion I
3,invincible,3,0.39,93.06,81.61,35.49,100.00,76.94,NaN,NaN,NaN,NaN
4,alex,4,0.52,92.74,96.74,50.29,73.14,71.97,ohAlex_#9493,Alex,NaN,NaN
5,luisito,5,0.58,90.12,90.12,71.77,71.03,43.47,Luisito#0412,Luisito,c:/Users/dmarc/OneDrive/Documents/Github/Perso...,NaN
6,steve,6,0.42,87.65,83.31,58.90,49.50,69.88,Steve#5376,Steve,c:/Users/dmarc/OneDrive/Documents/Github/Perso...,NaN
7,pacific_gull,7,0.52,86.49,84.32,64.00,59.60,46.71,NaN,NaN,NaN,NaN
8,greensleeves,8,0.55,84.46,83.21,34.47,83.65,41.12,NaN,NaN,NaN,NaN
9,muffled,9,0.44,83.80,73.17,37.21,68.15,59.91,NaN,NaN,NaN,NaN


# Helper Functions 

## Helper Functions for cleaning data from Ballchasing.com

In [4]:
def get_totals(df):
    # Select only the columns that contain the total counts for each statistic over the course of the season.
    total = df[[
    'team name', 'player name', 'games', 'wins', 'score', 'goals', 'assists', 'saves', 'shots', 'shots conceded', 'goals conceded',
    'goals conceded while last defender', 'amount collected', 'amount collected big pads', 'amount collected small pads',  
    'count collected big pads', 'count collected small pads','amount stolen', 'amount stolen big pads', 'amount stolen small pads',
    'count stolen big pads', 'count stolen small pads', '0 boost time', '100 boost time', 'amount used while supersonic',
    'amount overfill total','amount overfill stolen', 'total distance', 'time slow speed', 'time boost speed', 'time supersonic speed',
    'time on ground', 'time low in air', 'time high in air', 'time powerslide','count powerslide','time most back', 'time most forward',
    'time in front of ball','time behind ball', 'time defensive half', 'time offensive half', 'time defensive third', 'time neutral third',
    'time offensive third','demos inflicted', 'demos taken'
    ]]
    # Lowercase all playernames for consistency.
    total['player name'] = total["player name"].str.lower() 
    # Since we only have a dataframe of total statistics, we can group by player to account for BallChasing.com accidently
    # mismanaging 
    # team and player relationships. This is acting as a merge between rows where playernames match but the teams names didnt. 
    cleaned_data = total.groupby(['player name']).sum().reset_index()
    return cleaned_data

In [5]:
def average_player_statistics(cleaned_data):
    # In order to round the data to the nearest thousandth, we need to remove all columns without int datatypes. I divide the total
    # statistics for every player by the total number of (recorded) games that they have played and uploaded replays. 
    per_game_stats = cleaned_data.iloc[:,2:].div(cleaned_data.games, axis=0).round(4)
    # This is theseries that contains all columns without int datatypes, in other words all of the players. 
    players = cleaned_data.iloc[:,:2]
    # Then we merge these back together for the final dataframe.
    final_df = pd.concat([players, per_game_stats], axis=1)
    # return df so we can 
    return final_df

In [6]:
def remove_player_statistics(df, lst):
    # Provide as input a list of player names to be drop from this dataframe. 
    return df[~df['player name'].isin(lst)]

In [7]:
def patch_duplicates(df, dictionary):
    all_players = [list(dictionary.keys()), list(dictionary.values())]
    flattened = [item for sublist in all_players for item in sublist]
    # data = df.set_index('player name', drop = True, verify_integrity = True)
    data = df.set_index('player name', drop = True)
    # Remove players 
    patched_data = data.loc[flattened]
    patched_data = patched_data.rename(index=dictionary)
    patched_data = patched_data.reset_index(drop = False)\
        .groupby(['player name']).sum()\
            .reset_index(drop=False)
    # Removed players that we combined so we can recombine them
    removed_duplicates = remove_player_statistics(df, flattened).reset_index(drop = True)
    # Combine the two
    patched_dataframe = pd.concat([removed_duplicates, patched_data])
    return patched_dataframe.reset_index(drop=True)

## Helper Functions for generating player standings 

Current Season = TRL Winter 2021 Tournament

Win Rate

Offensive Rating
- score per game + shots per game + assists per game + shooting percentage + time most forward per game + time offensive half per game + time neurtal third per game + time offensive third per game

Defensive Rating
- score per game + saves per game + goals conceded per game + goals conceded while last defender per game + time most back per game + time defensive half per game + time neurtal third per game + time defensive third per game

Aggression
- shots per game + average speed per game + amount stolen per game + amount stolen big pads per game + amount stolen small pads per game + time offensive half per game + demos inflicted per game + demos taken per game + time powerslide per game + avg powerslide time per game + count powerslide per game

Speed
- bpm per game + avg boost amount per game + time supersonic per game +total distance per game + avg speed per game + average amount used while supersonic per game + amount collected per game + amount collected big pads per game + amount collected small pads per game + 0 boost time per game + 100 boost time per game + amount used while supersonic per game + time slow speed per game + time boost speed per game + time supersonic speed per game

In [8]:
def offensive_stats(df):
    # Select all columns with relevant data 
    Offensive = df[[
        'player name', 'score', 'shots', 'assists',
        'time most forward','time offensive half',
        'time neutral third','time offensive third'
        ]]
    # Create a new column with the calculated offensive rating value based on original column data calculations
    Offensive['Offensive Rating'] = .05*Offensive['score'] + .2*Offensive['shots'] \
                                + .05*Offensive['assists'] + .05*Offensive['time most forward'] + .05*Offensive['time neutral third'] \
                                + .1*Offensive['time offensive third'] 
    # The we use the MinMaxScaler from SKLearn to normalize all of the values and to generate ranks. 
    scaler = MinMaxScaler()
    Offensive['Offensive Rating']  = scaler.fit_transform(Offensive[['Offensive Rating']])
    # Return the DataFrame containing [player name, offensive rating] 
    # Columns in sorted order with respect to Offensive rating  
    return Offensive.sort_values('Offensive Rating',ascending=False)[['player name', 'Offensive Rating']]

In [9]:
def defensive_stats(df):
    # Select all columns with relevant data 
    Defensive = df[[
        'player name', 'score', 'saves','goals conceded','goals conceded while last defender',
        'time most back','time defensive half','time neutral third','time defensive third'
        ]]
    # Create a new column with the calculated defensive rating value based on original column data calculations
    Defensive['Defensive Rating'] = .05* Defensive['score']  + .2*Defensive['saves'] + .05*Defensive['goals conceded'] +\
                                    .1*Defensive['goals conceded while last defender']+ .1*Defensive['time most back'] +\
                                    .15*Defensive['time defensive half'] + .1 * Defensive['time neutral third'] +\
                                    .05 * Defensive['time defensive third']
    # The we use the MinMaxScaler from SKLearn to normalize all of the values and to generate ranks. 
    scaler = MinMaxScaler()
    Defensive['Defensive Rating']  = scaler.fit_transform(Defensive[['Defensive Rating']])
    # Return the DataFrame containing [player name, defensive rating] 
    # Columns in sorted order with respect to Offensive rating  
    return Defensive.sort_values('Defensive Rating',ascending=False)[['player name', 'Defensive Rating']]

In [10]:
def aggression_stats(df):
    # Select all columns with relevant data 
    Aggression = df[[
        'player name', 'shots', 'amount stolen', 'amount stolen big pads', 'amount stolen small pads',
        'time offensive half','demos inflicted','demos taken','time powerslide','count powerslide'
        ]]
    # Create a new column with the calculated defensive rating value based on original column data calculations
    Aggression['Aggression Rating'] = .2*Aggression['shots'] +.025*Aggression['amount stolen']+ .05*Aggression['amount stolen big pads']+\
                                        .025*Aggression['amount stolen small pads'] + .075*Aggression['time offensive half'] +\
                                        .2*Aggression['demos inflicted'] + .025*Aggression['demos taken'] +\
                                        .05*Aggression['time powerslide'] + .15*Aggression['count powerslide']
    # The we use the MinMaxScaler from SKLearn to normalize all of the values and to generate ranks.
    scaler = MinMaxScaler()
    Aggression['Aggression Rating']  = scaler.fit_transform(Aggression[['Aggression Rating']])
    # Return the DataFrame containing [player name, aggression rating] 
    # Columns in sorted order with respect to aggression rating  
    return Aggression.sort_values('Aggression Rating',ascending=False)[['player name', 'Aggression Rating']]

In [11]:
def speed_stats(df):
    # Select all columns with relevant data 
    Speed = df[[
        'player name','total distance','amount collected','amount collected big pads','amount collected small pads',
        '0 boost time','100 boost time','amount used while supersonic','time slow speed','time boost speed','time supersonic speed'
        ]]
    # Create a new column with the calculated defensive rating value based on original column data calculations
    Speed['Speed Rating'] = + 4*Speed['total distance']\
                        + (-3*Speed['amount used while supersonic']) + 1*Speed['amount collected']\
                        + 1*Speed['amount collected big pads'] + 2*Speed['amount collected small pads'] + (-3*Speed['0 boost time'])\
                        + 3*Speed['100 boost time'] + -1*Speed['time slow speed'] + 1*Speed['time boost speed']\
                        + 3*Speed['time supersonic speed']
    # The we use the MinMaxScaler from SKLearn to normalize all of the values and to generate ranks.
    scaler = MinMaxScaler()
    # Return the DataFrame containing [player name, speed rating] 
    # Columns in sorted order with respect to speed rating  
    Speed['Speed Rating'] = scaler.fit_transform(Speed[['Speed Rating']])
    return Speed.sort_values('Speed Rating',ascending=False)[['player name', 'Speed Rating']]

## Helper Functions to Generate Final Overall Standings DataFrame

In [12]:
def translate(value, leftMin, leftMax, rightMin, rightMax):
        """ Mapping a range of values to another """ 

        # Figure out how 'wide' each range is
        leftSpan = leftMax - leftMin
        rightSpan = rightMax - rightMin
        
        # Convert the left range into a 0-1 range (float)
        valueScaled = float(value - leftMin) / float(leftSpan)
        
        # Convert the 0-1 range into a value in the right range.
        return rightMin + (valueScaled * rightSpan)

In [13]:
def generate_overall_standings(df, offensive_stats, defensive_stats, aggression_stats, speed_stats):
    # Take name and win rate from cleaned dataframe
    final_df = df[['player name', 'wins']]
    # Add the offensive player ratings to final df by merging on playername.
    final_df = final_df.merge(offensive_stats, how='left')
    # Add the defensive player ratings to final df by merging on playername.
    final_df = final_df.merge(defensive_stats, how='left')
    # Add the aggression player ratings to final df by merging on playername.
    final_df = final_df.merge(aggression_stats, how='left')
    # Add the speed player ratings to final df by merging on playername.
    final_df = final_df.merge(speed_stats, how='left')


    feature_weights = {
        'off': .25, 
        'def': .25, 
        'agro': .25, 
        'spdy': .25
    }
    # Formatting by reordering columns in a readable format. 
    overalls = final_df[['player name','wins','Offensive Rating', 'Defensive Rating', 'Aggression Rating', 'Speed Rating']]

    # Generate overall standings based on all generated stats
    overalls['Total Overall'] = feature_weights['off']*overalls['Offensive Rating'] +\
                                feature_weights['def']*overalls['Defensive Rating'] +\
                                feature_weights['agro']*overalls['Aggression Rating'] +\
                                feature_weights['spdy']*overalls['Speed Rating']
    
    # Scaler for normalization                             
    scaler = MinMaxScaler()
    # Normalize
    overalls['Total Overall'] = scaler.fit_transform(overalls[['Total Overall']])
    # Sort values by total overall score
    total_overalls = overalls.sort_values('Total Overall',ascending=False)

    # Formatting for easier human readability
    total_overalls.reset_index(drop=True).round(3)
    total_overalls['Offensive Rating'] = total_overalls['Offensive Rating'] *100 
    total_overalls['Defensive Rating'] = total_overalls['Defensive Rating'] *100 
    total_overalls['Aggression Rating'] = total_overalls['Aggression Rating'] *100 
    total_overalls['Speed Rating'] = total_overalls['Speed Rating'] *100 
    total_overalls['Total Overall'] = total_overalls['Total Overall'] *100 

    # Display the final dataframe for people to inspect
    results = total_overalls[[
        'player name', 'wins', 'Total Overall',
        'Offensive Rating', 'Defensive Rating', 'Aggression Rating', 'Speed Rating']]\
            .round(2).reset_index(drop=True)\
                .rename(columns={
                    "player name": "participant",
                    'wins': 'win rate',
                    "Total Overall": "Overall",
                    'Offensive Rating': 'Offense',
                    'Defensive Rating': 'Defense',
                    'Aggression Rating':'Aggression',
                    'Speed Rating': 'Speed'})

    return results

In [45]:
def add_relevant_metadata(standings, names_fp, playercard_imgs, rank_fp):
    df = standings.set_index('participant')

    # Merge the player specified names
    player_names_fp = r'.\output\playercard_names.csv'
    player_names_df = pd.read_csv(player_names_fp, index_col=0)
    player_names_df = player_names_df.drop_duplicates(subset=['message_author', 'leaderboard_name'], keep='last')
    cols = {'message_author': 'discord_username', 'leaderboard_name': 'ballchasing_username'}
    names_df = player_names_df.rename(columns = cols)
    names_df.ballchasing_username = names_df.ballchasing_username.str.lower()
    names_df= names_df.set_index('ballchasing_username', drop = True)
    merged_names = df.merge(names_df, left_index = True, right_index = True, how = 'left')
    merged_names = merged_names.reset_index(drop = False) # Reset the index here to preserve original playercard rank data
    old_index = {'index': 'name_index'}
    merged_names = merged_names.rename(columns = old_index)

    # Merge the playercard image paths 
    x = os.listdir("./assets/playercard_imgs")
    start_directory = 'c:\\Users\\dmarc\\OneDrive\\Documents\\Github\\Personal Projects\\TritonRL-ScoreBot'
    abs_path = start_directory + '\\playercard_imgs\\'
    file_paths = [abs_path + i for i in x]
    names = [name.strip('.png') for name in x]
    fp_df = {'avatar_username': names, 'img_filepath': file_paths}
    fp_df = pd.DataFrame(data = fp_df)
    merged_names_paths = merged_names.merge(fp_df,  left_on = 'discord_username', right_on='avatar_username', how = 'left')

    # Merge the players associated rank 
    player_ranks_fp = r'.\output\playercard_ranks.csv'
    player_ranks_df = pd.read_csv(player_ranks_fp, index_col=0)
    player_ranks_df = player_ranks_df.drop_duplicates(subset=['message_author'], keep='last')
    merged_names_paths_ranks = merged_names_paths.merge(player_ranks_df, left_on = 'discord_username', right_on='message_author', how = 'left')

    # Clean Final DataFrame
    cleaned_df = merged_names_paths_ranks.sort_values('Overall', ascending = False)
    cleaned_df = cleaned_df.reset_index(drop=True) # Remove old index 
    cleaned_df = cleaned_df.reset_index(drop=False) # make a new index 
    cleaned_df = cleaned_df.set_index('name_index', drop = True)
    cleaned_df = cleaned_df.fillna('N/A')
    cleaned_df['img_filepath'] = cleaned_df['img_filepath'].str.replace('\\', '/')
    cleaned_df = cleaned_df.fillna('N/A')
    cleaned_df = cleaned_df.drop(columns = ['avatar_username', 'message_author'])

    return cleaned_df

# Load in Fall 2020 player data

In [46]:
fall_2020 = 'data\TRL_F20_playerdata_final.csv'
df = pd.read_csv(fall_2020)
fall_2020_totals = get_totals(df)
fall_2020_averages = average_player_statistics(fall_2020_totals)

# Load in Data

In [54]:
winter_2021 = 'data\wk4_updated_data.csv'
winter_2021 = pd.read_csv(winter_2021)
winter_2021_totals = get_totals(winter_2021)

In [56]:
winter_2021_totals.head()

,player name,games,wins,score,goals,assists,saves,shots,shots conceded,goals conceded,goals conceded while last defender,amount collected,amount collected big pads,amount collected small pads,count collected big pads,count collected small pads,amount stolen,amount stolen big pads,amount stolen small pads,count stolen big pads,count stolen small pads,0 boost time,100 boost time,amount used while supersonic,amount overfill total,amount overfill stolen,total distance,time slow speed,time boost speed,time supersonic speed,time on ground,time low in air,time high in air,time powerslide,count powerslide,time most back,time most forward,time in front of ball,time behind ball,time defensive half,time offensive half,time defensive third,time neutral third,time offensive third,demos inflicted,demos taken
0,adamamango,67,26,21358,43,32,83,137,486,175,67,128361,87344,41017,1049,3499,27399,17307,10092,207,860,2686.33,2027.90,8486,16367,2838,32860428,12846.88,9851.90,1901.73,14779.83,9064.51,756.15,438.80,2696,9232.3,7575.6,6911.70,17688.78,16345.17,8255.27,11939.44,7985.17,4675.88,27,59
1,alex,63,33,30129,83,37,88,246,446,146,45,145979,112824,33155,1368,2912,33471,23057,10414,284,905,2298.69,2126.39,16459,24635,5451,33818476,9629.95,9508.55,3526.84,11843.90,9737.76,1083.68,436.23,4307,6276.6,8640.3,5795.72,16869.61,14378.13,8287.24,10984.91,6601.16,5079.28,91,61
2,brictone,57,27,15848,40,27,46,122,444,146,45,122168,75209,46959,912,4153,28847,15916,12931,193,1128,2083.14,1387.86,13248,13095,2490,29137236,10031.55,8881.31,1902.61,13485.23,6708.24,622.05,392.59,3187,6506.8,7256.5,6409.16,14406.29,13462.13,7353.21,9789.70,6726.75,4299.00,34,60
3,casillas361,63,33,10548,24,18,45,71,446,146,38,119399,84329,35070,968,2944,14988,7945,7043,97,606,4507.76,1821.16,26341,12324,1795,31387324,11183.61,9584.69,1943.93,15654.86,6614.91,442.49,513.78,4587,7321.8,6907.2,6026.95,16685.35,16125.37,6586.86,12271.67,6765.23,3675.40,28,44
4,chyaboi,54,33,19285,51,29,57,145,386,125,42,134108,98837,35271,1158,2954,25517,15797,9720,188,811,2650.87,1874.31,17114,12948,1998,27737736,8845.70,7788.17,2681.26,10926.92,7555.43,832.73,431.85,3968,5918.0,6287.2,5374.54,13940.61,12437.72,6877.30,9194.53,6092.87,4027.72,50,50


# Calculate Overalls 

In [49]:
# Load in Data 
winter_2021_totals = get_totals(winter_2021)

# Make quick changes as needed
name_changes = {
    'invincibleblaze': 'invincible',
    'nsdlakers4': 'shaunch', 
    'monkensteinr': 'monkenstein', 
    'minimy_ugf': 'minimug'
    } 
    
players_to_drop = [
    'squishy', 
    'tag cramification', 
    'yegs',
    'desolation'
    ]#, 'goofy']

In [50]:
# Using the name changes, merge the totals for data in each column
patched_totals = patch_duplicates(winter_2021_totals, name_changes)
# Calculate the average for each statistic 
winter_2021_averages = average_player_statistics(patched_totals)
# Remove any remaining unwanted players by name
trl_winter_players = remove_player_statistics(winter_2021_averages, players_to_drop)

# Calculate Replay Statistics 
offensive_df = offensive_stats(trl_winter_players)
defensive_df = defensive_stats(trl_winter_players)
aggression_df = aggression_stats(trl_winter_players)
speed_df = speed_stats(trl_winter_players)

# Calculate Standings 
standings = generate_overall_standings(trl_winter_players, offensive_df, defensive_df, aggression_df, speed_df)

# Scale the overalls to be between the interval of 58 - 99
standings['Overall'] = pd.Series([translate(ovr, 0, 100, 58, 99) for ovr in standings.Overall]).round(2)

In [51]:
standings

,participant,win rate,Overall,Offense,Defense,Aggression,Speed
0,ix mini,0.53,99.00,100.00,58.41,75.58,95.73
1,smiggy,0.58,95.97,96.81,93.02,60.72,61.00
2,shaunch,0.61,94.38,81.73,53.93,79.75,86.59
3,invincible,0.39,93.06,81.61,35.49,100.00,76.94
4,alex,0.52,92.74,96.74,50.29,73.14,71.97
5,luisito,0.58,90.12,90.12,71.77,71.03,43.47
6,steve,0.42,87.65,83.31,58.90,49.50,69.88
7,pacific_gull,0.52,86.49,84.32,64.00,59.60,46.71
8,greensleeves,0.55,84.46,83.21,34.47,83.65,41.12
9,muffled,0.44,83.80,73.17,37.21,68.15,59.91


In [57]:
player_names = 'output/playercard_names.csv'
player_imgs_dir = './assets/playercard_imgs'
player_ranks = 'output/playercard_ranks.csv'
playercard_data = add_relevant_metadata(standings, player_names, player_imgs_dir, player_ranks)
playercard_data


,index,win rate,Overall,Offense,Defense,Aggression,Speed,discord_username,playercard_name,img_filepath,rank
name_index,,,,,,,,,,,
ix mini,0,0.53,99.00,100.00,58.41,75.58,95.73,Mini#6984,IX Mini,c:/Users/dmarc/OneDrive/Documents/Github/Perso...,N/A
smiggy,1,0.58,95.97,96.81,93.02,60.72,61.00,Smiggy#1206,smiggy,c:/Users/dmarc/OneDrive/Documents/Github/Perso...,Grand Champion I
shaunch,2,0.61,94.38,81.73,53.93,79.75,86.59,Shaunch#6025,Shaunch,c:/Users/dmarc/OneDrive/Documents/Github/Perso...,Champion I
invincible,3,0.39,93.06,81.61,35.49,100.00,76.94,N/A,N/A,N/A,N/A
alex,4,0.52,92.74,96.74,50.29,73.14,71.97,ohAlex_#9493,Alex,N/A,N/A
luisito,5,0.58,90.12,90.12,71.77,71.03,43.47,Luisito#0412,Luisito,c:/Users/dmarc/OneDrive/Documents/Github/Perso...,N/A
steve,6,0.42,87.65,83.31,58.90,49.50,69.88,Steve#5376,Steve,c:/Users/dmarc/OneDrive/Documents/Github/Perso...,N/A
pacific_gull,7,0.52,86.49,84.32,64.00,59.60,46.71,N/A,N/A,N/A,N/A
greensleeves,8,0.55,84.46,83.21,34.47,83.65,41.12,N/A,N/A,N/A,N/A


In [58]:
# Save the new dataframe as Player Statistics 
playercard_data.to_csv('output/playercard_stats.csv', index=True)

In [36]:
# Processing for playercard_name function

# player_names_fp = r'.\output\playercard_names.csv'
# df = pd.read_csv(player_names_fp, index_col=0)
# df = df.drop_duplicates(subset=['message_author', 'leaderboard_name'], keep='last')
# df = df.set_index('message_author')

# try:
#     msg_author = 'ohAlex_#8725'
#     ballchasing_name = df.loc[msg_author]['leaderboard_name']
#     playercard_name = df.loc[msg_author]['playercard_name']
# except:
#     # await message.channel.send('You must first use the "!playercard_name" command before viewing your playercard.')
#     print('You must first use the "!playercard_name" command before viewing your playercard.')


# output_df = df.reset_index()
# player_names_fp = r'.\output\playercard_names.csv'
# output_df.to_csv(player_names_fp)

# df

You must first use the "!playercard_name" command before viewing your playercard.


,leaderboard_name,playercard_name
message_author,,
test,test,test
Goofy#8725,goofy,GFY


In [19]:
# def add_relevant_metadata(standings, names_fp, playercard_imgs, rank_fp):
#     df = standings.set_index('participant')
#     # # Merge the player specified names
#     player_names_fp = r'.\output\playercard_names.csv'
#     player_names_df = pd.read_csv(player_names_fp, index_col=0)
#     player_names_df = player_names_df.drop_duplicates(subset=['message_author', 'leaderboard_name'], keep='last')
#     cols = {'message_author': 'discord_username', 'leaderboard_name': 'ballchasing_username'}
#     names_df = player_names_df.rename(columns = cols)
#     names_df.ballchasing_username = names_df.ballchasing_username.str.lower()
#     names_df= names_df.set_index('ballchasing_username', drop = True)
#     merged_names = df.merge(names_df, left_index = True, right_index = True, how = 'outer')
#     merged_names = merged_names.reset_index(drop = False) # Reset the index here to preserve original playercard rank data
#     old_index = {'index': 'name_index'}
#     merged_names = merged_names.rename(columns = old_index)

#     # Merge the playercard image paths 
#     x = os.listdir("./playercard_imgs")
#     start_directory = 'c:\\Users\\dmarc\\OneDrive\\Documents\\Github\\Personal Projects\\TritonRL-ScoreBot'
#     abs_path = start_directory + '\\playercard_imgs\\'
#     file_paths = [abs_path + i for i in x]
#     names = [name.strip('.png') for name in x]
#     fp_df = {'avatar_username': names, 'img_filepath': file_paths}
#     fp_df = pd.DataFrame(data = fp_df)
#     merged_names_paths = merged_names.merge(fp_df,  left_on = 'discord_username', right_on='avatar_username', how = 'outer')

#     # Merge the players associated rank 
#     player_ranks = 'output/player_ranks.json'
#     def load_params(fp):
#         with open(fp) as fh:
#             param = json.load(fh)
#         return param
#     ranks = load_params(player_ranks)
#     cols = {'index': 'rank_username', 0: 'rank'}
#     rank_df = pd.DataFrame.from_dict(ranks,orient='index').reset_index(drop = False).rename({'index': 'player', '0': 'rank'})
#     rank_df = rank_df.rename(columns = cols)
#     merged_names_paths_ranks = merged_names_paths.merge(rank_df, left_on = 'discord_username', right_on='rank_username', how = 'outer')

#     # Clean Final DataFrame
#     cleaned_df = merged_names_paths_ranks.sort_values('Overall', ascending = False)
#     cleaned_df = cleaned_df.reset_index(drop=True) # Remove old index 
#     cleaned_df = cleaned_df.reset_index(drop=False) # Add new column based on dataframe ranked by Overall.
#     cleaned_df = cleaned_df.rename(columns={'index': 'ranking'})
#     cleaned_df = cleaned_df.fillna('N/A')
#     cleaned_df['img_filepath'] = cleaned_df['img_filepath'].str.replace('\\', '/')
#     cleaned_df = cleaned_df.drop(columns = ['avatar_username', 'rank_username']).reset_index(drop = True).set_index('name_index')

#     return cleaned_df